<a href="https://colab.research.google.com/github/Bolorooo24/CV-ML_labs/blob/main/Lab_2/Task_2/ResNet50_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
batch_size = 64

In [8]:
import torchvision.datasets as datasets
dataset = datasets.CIFAR100(root='./cifar100', train=True, download=True, transform=transform)

100%|██████████| 169M/169M [00:05<00:00, 31.0MB/s]


Extracting ./cifar100/cifar-100-python.tar.gz to ./cifar100


In [9]:
# put dataset structure into dictionary
classes_dict  = {}

for img, label in dataset:
  if label not in classes_dict:
    classes_dict[label] = []
  classes_dict[label].append(img)

In [18]:
# create subset of the dataset
import random
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def subset_create(classes_dict, num_classes, num_samples):
    sample_classes = random.sample(list(classes_dict.keys()), num_classes)
    train_data, test_data, train_labels, test_labels = [], [], [], []
    class_mapping = {cls: idx for idx, cls in enumerate(sample_classes)}

    for c in sample_classes:
        sample_images = random.sample(classes_dict[c], num_samples)

        train_imgs, test_imgs = train_test_split(sample_images, test_size=0.2, random_state=42)
        train_data.extend(train_imgs)
        train_labels.extend([class_mapping[c]] * len(train_imgs))
        test_data.extend(test_imgs)
        test_labels.extend([class_mapping[c]] * len(test_imgs))

    train_dataset = TensorDataset(torch.stack(train_data), torch.tensor(train_labels))
    test_dataset = TensorDataset(torch.stack(test_data), torch.tensor(test_labels))
    return train_dataset, test_dataset

In [20]:
# create each subset

subsets = {
    "10x10": subset_create(classes_dict, 10, 10),
    "10x100": subset_create(classes_dict, 10, 100),
    "10x200": subset_create(classes_dict, 10, 200),
    "100x10": subset_create(classes_dict, 100, 10),
    "100x100": subset_create(classes_dict, 100, 100),
    "100x200": subset_create(classes_dict, 100, 200),
}
print("subset created!!")

for key in subsets:
    train_subset, test_subset = subsets[key]
    subsets[key] = (train_subset, test_subset)

subset created!!


In [12]:
class ResNet50CustomInput(nn.Module):
    def __init__(self, classes, input_channels):
        super(ResNet50CustomInput, self).__init__()

        # Load the pre-trained ResNet-50 model without the final classification layer
        self.resnet = models.resnet50(pretrained=True)
        # Remove the original fully connected layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])

        # Modify the first convolution layer to accept smaller input
        self.resnet[0] = nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)

        # Adaptive average pooling layer to adapt to different input sizes
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        # New fully connected layer for your specific number of classes
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x





In [13]:
# train function on multiple subsets
def train(model, train_loader, optimizer, criterion, epochs):
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    for i, labels in train_loader:
      i, labels = i.to(device), labels.to(device)
      optimizer.zero_grad()
      outputs = model(i)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss/len(train_loader):.4f}")





In [14]:
def test(model, test_loader):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for i, labels in test_loader:
      i, labels = i.to(device), labels.to(device)
      outputs = model(i)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  return 100 * correct / total

In [15]:
device = torch.device("cpu")


In [16]:
for subset_name, subset in subsets.items():
    print(f"Subset {subset_name} labels: {set(subset.tensors[1].tolist())}")

Subset 10x10 labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Subset 10x100 labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Subset 10x200 labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Subset 100x10 labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}
Subset 100x100 labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}
Subset 100x200 

In [ ]:
from torchvision import models

results = {}
epochs = 50
input_channels = 3

for subset_name, (train_subset, test_subset) in subsets.items():
  print(f"Training on subset: {subset_name}")
  num_classes = len(set(train_subset.tensors[1].tolist()))
  train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

  model = ResNet50CustomInput(classes=num_classes, input_channels=input_channels).to(device)
  optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()

  train(model, train_loader, optimizer, criterion, epochs)
  accuracy = test(model, test_loader)
  print(f"Accuracy of {subset_name}: {accuracy:.2f}%")
  torch.save(model.state_dict(), f"/content/drive/MyDrive/models/ResNet50/{subset_name}_ResNet50_model.pth")
  results[subset_name] = accuracy
print("\nFinal results:")
for subset_name, accuracy in results.items():
    print(f"{subset_name}: {accuracy:.2f}%")

Training on subset: 10x10


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 2.3245
Epoch 2, Loss: 2.1566
Epoch 3, Loss: 1.9241
Epoch 4, Loss: 1.8352
Epoch 5, Loss: 1.5677
Epoch 6, Loss: 1.3659
Epoch 7, Loss: 1.3088
Epoch 8, Loss: 1.1237
Epoch 9, Loss: 1.0896
Epoch 10, Loss: 0.9663
Epoch 11, Loss: 0.7070
Epoch 12, Loss: 0.7008
Epoch 13, Loss: 0.5976
Epoch 14, Loss: 0.4648
Epoch 15, Loss: 0.5973
Epoch 16, Loss: 0.3577
Epoch 17, Loss: 0.3151
Epoch 18, Loss: 0.3775
Epoch 19, Loss: 0.2741
Epoch 20, Loss: 0.2889
Epoch 21, Loss: 0.2738
Epoch 22, Loss: 0.2337
Epoch 23, Loss: 0.1427
Epoch 24, Loss: 0.1027
Epoch 25, Loss: 0.1561
Epoch 26, Loss: 0.1616
Epoch 27, Loss: 0.1178
Epoch 28, Loss: 0.1041
Epoch 29, Loss: 0.0830
Epoch 30, Loss: 0.1645
Epoch 31, Loss: 0.1663
Epoch 32, Loss: 0.0681
Epoch 33, Loss: 0.0780
Epoch 34, Loss: 0.0864
Epoch 35, Loss: 0.0494
Epoch 36, Loss: 0.0736
Epoch 37, Loss: 0.0668
Epoch 38, Loss: 0.0623
Epoch 39, Loss: 0.0645
Epoch 40, Loss: 0.0692
Epoch 41, Loss: 0.0441
Epoch 42, Loss: 0.0337
Epoch 43, Loss: 0.0407
Epoch 44, Loss: 0.03

In [ ]:
for subset_name, accuracy in results.items():
    print(f"subset {subset_name}: Accuracy = {accuracy:.2f}%")

In [ ]:
import numpy as np


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

subsets = {
    "10x10": 10,
    "10x100": 10,
    "10x200": 10,
    "100x10": 100,
    "100x100": 100,
    "100x200": 100,
}

dataset = datasets.CIFAR100(root='./cifar100', train=True, download=True)
image, label = dataset[0]
classes = dataset.classes
image_np = np.array(image)
image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
input_image = transform(image_np)
input_image = input_image.unsqueeze(0).to(device)

for subset_name, num_classes in subsets.items():
    model = Net(classes=num_classes).to(device)
    model.load_state_dict(torch.load(f"{subset_name}_model.pth"))
    model.eval()
    with torch.no_grad():
        outputs = model(input_image)
        _, predicted = torch.max(outputs, 1)
    predicted_class = classes[predicted.item()]
    confidence = torch.softmax(outputs, dim=1)[0][predicted.item()].item()
    print(f"Model: {subset_name}")
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}%")
    plt.imshow(image_np)
    plt.title(f'Model: {subset_name}\nPredicted Class: {predicted_class}\nConfidence: {confidence:.2f}')
    plt.axis('off')
    plt.show()

